# Local Torrent Downloader with Auto-Archive

This script downloads torrents to local Colab storage (~70GB) and automatically creates 700MB archives for easy download.

⚠️ **Important**: For research and educational purposes only. Please respect copyright laws and use only for legal content.

## 1. Install Dependencies

In [ ]:
# Install dependencies
!apt-get update
!apt-get install -y libtorrent-rasterbar-dev p7zip-full
!pip install libtorrent

# Check installation
import libtorrent as lt
print(f"✅ LibTorrent version: {lt.version}")
print("✅ 7zip installed for archiving")
print("✅ Installation completed successfully!")
print(f"📁 Available local storage: ~70GB in /content/")

## 2. Setup Local Storage

In [ ]:
import os
import shutil

# Create universal directories
downloads_torrent = '/content/downloads_torrent'  # All downloads go here
archive_dir = '/content/archives'

os.makedirs(downloads_torrent, exist_ok=True)
os.makedirs(archive_dir, exist_ok=True)

# Check available space
def get_disk_usage(path):
    total, used, free = shutil.disk_usage(path)
    return {
        'total': total // (1024**3),  # GB
        'used': used // (1024**3),
        'free': free // (1024**3)
    }

usage = get_disk_usage('/content')
print(f"💾 Disk usage: {usage['used']}GB used / {usage['total']}GB total")
print(f"💾 Available space: {usage['free']}GB")
print(f"📁 Downloads directory: {downloads_torrent}")
print(f"📦 Archive directory: {archive_dir}")

## 3. Enhanced Torrent Downloader with Auto-Archive

In [ ]:
import libtorrent as lt
import time
import os
import subprocess
import shutil
from IPython.display import clear_output
from google.colab import files
import datetime

class UniversalTorrentDownloader:
    def __init__(self, downloads_path, archive_path):
        self.downloads_path = downloads_path
        self.archive_path = archive_path
        self.session = lt.session()
        self.session.listen_on(6881, 6891)
        
    def download_torrent(self, torrent_file_or_magnet, auto_archive=True, auto_download=True):
        """Download torrent, archive, and download archives automatically"""
        try:
            # Check if it's a magnet link or file
            if torrent_file_or_magnet.startswith('magnet:'):
                handle = lt.add_magnet_uri(self.session, torrent_file_or_magnet, {
                    'save_path': self.downloads_path
                })
            else:
                # Load torrent file
                info = lt.torrent_info(torrent_file_or_magnet)
                handle = self.session.add_torrent({
                    'ti': info,
                    'save_path': self.downloads_path
                })
            
            torrent_name = handle.name()
            print(f"🚀 Starting download: {torrent_name}")
            
            # Progress monitoring
            while not handle.is_seed():
                status = handle.status()
                
                clear_output(wait=True)
                print(f"📁 Torrent: {torrent_name}")
                print(f"📊 Progress: {status.progress * 100:.1f}%")
                print(f"⬇️ Download: {status.download_rate / 1000:.1f} KB/s")
                print(f"⬆️ Upload: {status.upload_rate / 1000:.1f} KB/s")
                print(f"👥 Peers: {status.num_peers} | Seeds: {status.num_seeds}")
                
                # Show disk usage
                usage = get_disk_usage('/content')
                print(f"💾 Disk: {usage['used']}GB used / {usage['free']}GB free")
                
                if status.state == lt.torrent_status.downloading:
                    print("🔄 Status: Downloading")
                elif status.state == lt.torrent_status.finished:
                    print("✅ Status: Download Finished")
                    break
                elif status.state == lt.torrent_status.seeding:
                    print("🌱 Status: Seeding")
                    break
                
                time.sleep(2)
            
            print(f"\n✅ Download completed: {torrent_name}")
            
            # Auto-archive if requested
            if auto_archive:
                print("📦 Creating archives...")
                self.create_archives_from_downloads()
                
                # Auto-download archives if requested
                if auto_download:
                    print("📥 Starting automatic download...")
                    self.download_all_archives()
                
        except Exception as e:
            print(f"❌ Download error: {e}")
    
    def create_archives_from_downloads(self):
        """Archive everything in downloads_torrent folder"""
        try:
            if not os.path.exists(self.downloads_path):
                print(f"❌ Downloads path not found: {self.downloads_path}")
                return
            
            # Check if there are any files to archive
            files_to_archive = []
            for root, dirs, files in os.walk(self.downloads_path):
                for file in files:
                    files_to_archive.append(os.path.join(root, file))
            
            if not files_to_archive:
                print("❌ No files to archive in downloads folder")
                return
            
            # Create timestamp for unique archive name
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            archive_name = os.path.join(self.archive_path, f"torrent_archive_{timestamp}.7z")
            
            print(f"📦 Creating archive: torrent_archive_{timestamp}.7z")
            print(f"📁 Source: {self.downloads_path}")
            print(f"📄 Files to archive: {len(files_to_archive)}")
            
            # Create 700MB volumes using 7zip
            cmd = [
                '7z', 'a',
                '-v700m',  # 700MB volumes
                '-mx=5',   # Medium compression
                archive_name,
                f"{self.downloads_path}/*"
            ]
            
            print("🔄 Compressing... This may take a while.")
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0:
                print("✅ Archive created successfully!")
                self.list_archives()
                
            else:
                print(f"❌ Archive creation failed: {result.stderr}")
                
        except Exception as e:
            print(f"❌ Archive error: {e}")
    
    def list_downloads(self):
        """Show downloaded files"""
        print("\n📁 Downloaded files:")
        if not os.path.exists(self.downloads_path):
            print("No downloads yet.")
            return
            
        total_size = 0
        for root, dirs, files in os.walk(self.downloads_path):
            level = root.replace(self.downloads_path, '').count(os.sep)
            indent = ' ' * 2 * level
            folder_name = os.path.basename(root) if level > 0 else "downloads_torrent"
            print(f"{indent}📁 {folder_name}/")
            
            sub_indent = ' ' * 2 * (level + 1)
            for file in files:
                file_path = os.path.join(root, file)
                size = os.path.getsize(file_path)
                size_mb = size / (1024 * 1024)
                total_size += size
                print(f"{sub_indent}📄 {file} ({size_mb:.1f} MB)")
        
        print(f"\n💾 Total size: {total_size / (1024**3):.2f} GB")
    
    def list_archives(self):
        """Show created archives"""
        print("\n📦 Created archives:")
        if not os.path.exists(self.archive_path):
            print("No archives yet.")
            return
            
        all_files = os.listdir(self.archive_path)
        archive_files = [f for f in all_files if '.7z.' in f or f.endswith('.7z')]
        
        if not archive_files:
            print("No archive files found.")
            return
        
        # Group by base name
        archive_groups = {}
        for file in archive_files:
            if '.7z.' in file:
                base_name = file.split('.7z.')[0]
            else:
                base_name = file.replace('.7z', '')
            
            if base_name not in archive_groups:
                archive_groups[base_name] = []
            archive_groups[base_name].append(file)
        
        total_size = 0
        for base_name, volumes in archive_groups.items():
            volumes.sort(key=lambda x: int(x.split('.7z.')[-1]) if '.7z.' in x else 0)
            print(f"\n📦 {base_name} ({len(volumes)} volumes):")
            
            for volume in volumes:
                volume_path = os.path.join(self.archive_path, volume)
                size = os.path.getsize(volume_path)
                size_mb = size / (1024 * 1024)
                total_size += size
                print(f"  📄 {volume} ({size_mb:.1f} MB)")
        
        print(f"\n💾 Total archive size: {total_size / (1024**3):.2f} GB")
        print(f"📊 Total volumes: {len(archive_files)}")
    
    def download_all_archives(self):
        """Download all archive files to local computer"""
        if not os.path.exists(self.archive_path):
            print("❌ Archive directory not found")
            return
        
        archive_files = [f for f in os.listdir(self.archive_path) if '.7z.' in f or f.endswith('.7z')]
        
        if not archive_files:
            print("❌ No archive files found")
            return
        
        # Sort files
        archive_files.sort()
        
        total_size = sum(os.path.getsize(os.path.join(self.archive_path, f)) for f in archive_files)
        total_size_gb = total_size / (1024**3)
        
        print(f"📥 Downloading {len(archive_files)} archive volumes")
        print(f"💾 Total size: {total_size_gb:.2f} GB")
        print("⚠️ Make sure you have enough space on your computer!")
        print()
        
        # Download each file
        for i, filename in enumerate(archive_files, 1):
            file_path = os.path.join(self.archive_path, filename)
            size_mb = os.path.getsize(file_path) / (1024*1024)
            print(f"⬇️ Downloading {i}/{len(archive_files)}: {filename} ({size_mb:.1f} MB)")
            
            try:
                files.download(file_path)
                print(f"✅ Downloaded: {filename}")
            except Exception as e:
                print(f"❌ Failed to download {filename}: {e}")
        
        print(f"\n🎉 Download complete! Downloaded {len(archive_files)} volumes")
        print("📝 Extract with 7zip, WinRAR, or similar tools")
        print("📝 All volumes must be in the same folder when extracting")
    
    def cleanup(self, keep_archives=True):
        """Clean up downloaded files"""
        if os.path.exists(self.downloads_path):
            shutil.rmtree(self.downloads_path)
            os.makedirs(self.downloads_path, exist_ok=True)
            print("🗑️ Download folder cleaned")
        
        if not keep_archives and os.path.exists(self.archive_path):
            shutil.rmtree(self.archive_path)
            os.makedirs(self.archive_path, exist_ok=True)
            print("🗑️ Archive folder cleaned")

# Create universal downloader instance
downloader = UniversalTorrentDownloader(downloads_torrent, archive_dir)
print("✅ Universal torrent downloader ready!")
print("📁 Downloads: /content/downloads_torrent/")
print("📦 Archives: /content/archives/")
print("🔄 Workflow: Download → Archive → Auto-download")

## 4. Download Torrent

In [ ]:
# Option 1: Download by magnet link
magnet_link = "magnet:?xt=urn:btih:HASH&dn=filename"  # Replace with your magnet link
# downloader.download_torrent(magnet_link)

# Option 2: Download by torrent file
print("📁 Select torrent file for download:")
uploaded = files.upload()

if uploaded:
    torrent_file = list(uploaded.keys())[0]
    print(f"📄 Uploaded file: {torrent_file}")
    
    # Start download with auto-archive
    downloader.download_torrent(torrent_file, auto_archive=True)
else:
    print("❌ No file uploaded")

## 5. View Archives and Download

In [ ]:
# Show all archives and disk usage
downloader.list_archives()

# Show current disk usage
usage = get_disk_usage('/content')
print(f"\n💾 Current disk usage: {usage['used']}GB / {usage['total']}GB")
print(f"💾 Available space: {usage['free']}GB")

## 6. Download Archives (if needed)

In [ ]:
# Download all archives (only if auto-download failed or you need to re-download)
print("📥 Re-downloading all archives...")
print("💡 Note: Archives are downloaded automatically after torrent completion")
print("🔧 Use this only if you need to download archives again")

downloader.download_all_archives()

## 7. Cleanup

In [ ]:
# Clean up downloaded files (keep archives)
# downloader.cleanup(keep_archives=True)

# Clean up everything
# downloader.cleanup(keep_archives=False)

print("🧹 Uncomment the lines above to clean up files")

## 📝 Notes

### Universal Workflow
1. **Download torrent** (Section 4) → Files go to `/content/downloads_torrent/`
2. **Auto-archive** → Creates 700MB volumes with timestamp
3. **Auto-download** → All archive volumes download to your computer

### Storage
- **70GB** local storage available in `/content/`
- Files are automatically archived into **700MB volumes**
- Archives can be downloaded individually if needed
- Storage is cleared when Colab session restarts

### Archive Format
- Uses 7zip compression with 700MB volumes
- Files named: `torrent_archive_TIMESTAMP.7z.001`, `.002`, etc.
- Extract with 7zip, WinRAR, or similar tools

### Advantages
- **Fully automatic**: Download → Archive → Download
- **Universal**: Works with any torrent, no name dependencies
- **More storage**: 70GB vs 15GB Google Drive
- **Faster downloads**: Local storage is faster
- **Easy archiving**: Auto-split into manageable chunks

### Limitations
- Files lost when session restarts (archives are downloaded first!)
- Individual file download limit: ~2GB via Colab
- Session timeout: 12 hours maximum

### Legal Notice
**For research and educational purposes only.** Use only for legal content and respect copyright laws.

In [ ]:
# Clean up downloaded files (keep archives)
# downloader.cleanup(keep_archives=True)

# Clean up everything
# downloader.cleanup(keep_archives=False)

print("🧹 Uncomment the lines above to clean up files")

## 📝 Notes

### Storage
- **70GB** local storage available in `/content/`
- Files are automatically archived into **700MB volumes**
- Archives can be downloaded individually
- Storage is cleared when Colab session restarts

### Archive Format
- Uses 7zip compression with 700MB volumes
- Files named: `archive.7z.001`, `archive.7z.002`, etc.
- Extract with 7zip, WinRAR, or similar tools

### Advantages over Google Drive version
- **More storage**: 70GB vs 15GB
- **Faster downloads**: Local storage is faster
- **Easy archiving**: Auto-split into manageable chunks
- **No cloud authorization**: Works without Drive mounting

### Limitations
- Files lost when session restarts (download archives first!)
- Individual file download limit: ~2GB via Colab
- Session timeout: 12 hours maximum

### Legal Notice
**For research and educational purposes only.** Use only for legal content and respect copyright laws.